We'll try to fetch the artists and tracks I listened to in a specific period from lastfm and see if we can encirh this info with spotify data (because it only allows to 50 last tracks in a beta api)
After that we can try som simple ML models on it :D 

In [1]:
import requests as re
#import numpy as np
import time
import pandas as pd
import datetime
from datetime import date, timedelta
import json
import requests_cache
from IPython.display import clear_output

### Pre-work

In [2]:
#Get the csv from the last request
#COLUMNS 0 -> BAND NAME
#COLUMNS 1 -> SONG NAME
#COLUMNS 2 -> PLAYED AT UNIX TIMESTAMP
tracks = pd.read_csv('played_tracks.csv')

In [3]:

tracks.drop(tracks.columns[2], axis = 1, inplace = True)
#clear duplicate columns
tracks = tracks.drop_duplicates()
#rename columns
tracks.rename(mapper= {'0': 'band_name', '1': 'song_name'}, axis = 1, inplace = True)

## Spotify API

In [4]:
#create a request cache
requests_cache.install_cache('track_features_cache')

In [5]:
#song id
#request.json()['tracks']['items'][0]['id']
#artist
#request.json()['tracks']['items'][0]['album']['artists'][0]['name']

In [71]:
#define a function for the api request that returns the json response
def search_song_request(_song_name, _band_name, i):
    r = re.get('https://api.spotify.com/v1/search?' + 'q=artist:' + _band_name + '%20track:' + _song_name + '&market:from_token' + '&type=track&limit=50&include_external=audio' , headers = headers)
    return r.json()['tracks']['items']

#define a function to make the api request, iterate on the response and get the song id based on the artist
def get_song_id(_song_name, _band_name):
    clear_output(wait=True)
    print("working on song", _song_name, "by", _band_name, "starting at", datetime.now())
    _response = search_song_request(_song_name, _band_name)
    for i in range(len(_response)):
        #there must be a better way of doing this byt it'll work for now: we iterate through the entire artist list - there is a lot of useless data in the response
        if _response[i]['album']['artists'][0]['name'] == _band_name:
            return _response[i]['id']
        else:
            pass
    #if no returns, we could not find id, so it returns not_found
    return 'not_found'


In [7]:
#Authentication - getting an access token
auth_response = re.post('https://accounts.spotify.com/api/token', {
    'grant_type': 'client_credentials',
    'client_id': '6cb782c1b0404843b3e5a06e8361cb6e',
    'client_secret': 'a32c28deab714fa89091395089cbca90',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#create a header with the access token
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [ ]:
print("start:", datetime.datetime.now())

In [79]:
tracks['sp_id'] = tracks.apply(lambda x: get_song_id(x['song_name'], x['band_name']), axis = 1)

Int64Index([1535, 11914, 3197, 8131, 7459, 7784, 6358, 9011, 9233, 10289], dtype='int64')

In [ ]:
print("saving file at", datetime.now())
tracks.to_csv('tracks_features.csv', index=False)
print("all finished at", datetime.now())